In [1]:
# import required libraries
import pandas as pd
import numpy as np
import random
import datetime
import sqlite3
import os

For this SQL tutorial I'm using a synthetic dataset created below. It consists of two tables, customers and orders. The customers table contains information about the customers, such as their name, region, and phone number. The orders table contains information about the orders, such as the order date, the customer id, and the total amount of the order.

In [2]:
# set random seed
random_seed = 42
np.random.seed(random_seed)
random.seed(random_seed)

In [3]:
# create dataframes
customers = pd.DataFrame({
    'customer_id': range(1, 51),
    'name': ['Customer ' + str(i) for i in range(1, 51)],
    'region': ['Region ' + str(random.randint(1, 5)) for i in range(50)],
    'phone_number': ['555-555-' + str(random.randint(1000, 9999)) for i in range(50)]
})

order_numbers = np.arange(1, 1001)
customer_ids = np.random.choice(customers['customer_id'], size=1000, replace=True)
dates = pd.date_range(start='2022-01-01', end='2022-12-31', periods=1000).strftime('%m/%d/%Y')
amounts = [random.randint(1, 1000) for i in range(1000)]
tracking_numbers = ['TRACK' + str(random.randint(10000, 99999)) for i in range(1000)]

orders = pd.DataFrame({
    'order_number': order_numbers,
    'customer_id': customer_ids,
    'date': dates,
    'amount': amounts,
    'tracking_number': tracking_numbers
})

In [4]:
# create SQLite database
db_filename = 'data/orders.db'
if not os.path.exists('data'):
    os.mkdir('data')

conn = sqlite3.connect(db_filename)

In [5]:
# copy dataframes to SQLite tables
customers.to_sql('customers', conn, index=False, if_exists='replace')
orders.to_sql('orders', conn, index=False, if_exists='replace')

# close database connection
conn.close()

In [3]:
%load_ext sql

In [4]:
%sql sqlite:///data/orders.db

'Connected: @data/orders.db'

In [23]:
%%sql 

select * from customers limit 10

 * sqlite:///data/orders.db
Done.


customer_id,name,region,phone_number
1,Customer 1,Region 1,555-555-5803
2,Customer 2,Region 1,555-555-6925
3,Customer 3,Region 3,555-555-4150
4,Customer 4,Region 2,555-555-2139
5,Customer 5,Region 2,555-555-1750
6,Customer 6,Region 2,555-555-4733
7,Customer 7,Region 1,555-555-5741
8,Customer 8,Region 5,555-555-2307
9,Customer 9,Region 1,555-555-4814
10,Customer 10,Region 5,555-555-2654


In [28]:
%%sql
select * from orders
where customer_id = 1

 * sqlite:///data/orders.db
Done.


order_number,customer_id,date,amount,tracking_number
98,1,02/05/2022,498,TRACK25525
103,1,02/07/2022,785,TRACK91057
134,1,02/18/2022,124,TRACK71592
140,1,02/20/2022,603,TRACK69510
186,1,03/09/2022,747,TRACK12719
224,1,03/23/2022,993,TRACK25194
265,1,04/07/2022,754,TRACK30181
281,1,04/13/2022,521,TRACK27892
415,1,05/31/2022,905,TRACK69878
467,1,06/19/2022,755,TRACK36318


In [5]:
%%sql
SELECT c.region, c.customer_id, c.name, SUM(o.amount) as order_sum
FROM customers c
INNER JOIN orders o ON c.customer_id = o.customer_id
WHERE substr(o.date, 1, 2) = '02'
GROUP BY c.region, c.customer_id, c.name
ORDER BY c.region, order_sum DESC


 * sqlite:///data/orders.db
Done.


region,customer_id,name,order_sum
Region 1,1,Customer 1,2010
Region 1,9,Customer 9,1405
Region 1,39,Customer 39,1379
Region 1,37,Customer 37,1163
Region 1,7,Customer 7,1023
Region 1,14,Customer 14,950
Region 1,49,Customer 49,881
Region 1,12,Customer 12,748
Region 1,44,Customer 44,512
Region 1,47,Customer 47,312
